In [109]:
import re
from collections import Counter
from pprint import pprint
from math import log

'''Word Probability'''
def words(text): return re.findall(r'\w+', text.lower())
count_word = Counter(words(open('big.txt').read()))
Nw = sum(count_word.values())
Pdist = {word: float(count)/Nw for word, count in count_word.items()}

In [110]:
def correction(word):
    MAXBEAM = 500
    total_state = []
    states = [("", word, 0, Pw(word),1)]
    for i in range(len(word)):
#         print(i, states[:3])
        new_s = []
        for s in states:
            new_s += next_states(s)
        states = new_s
        d = {}
        for i in states:
            if i[0]+i[1] in d:
                if d[i[0]+i[1]][2] > i[2]:
                    d[i[0]+i[1]] = i
            else:
                d[i[0]+i[1]] = i
        states = d.values()
        states = sorted(states, key=lambda x: x[2]) 
        states = sorted(states, key=lambda x: P(x[4],x[3],x[2]),reverse=True)[:MAXBEAM] 
#         pprint(states[:3])
#     return max(states[:3])
    return states[:10]

In [111]:
def next_states(state):
    letters    = 'abcdefghijklmnopqrstuvwxyz'
    L, R, edit, pw, pedit = state
    R0, R1 = R[0], R[1:]
    if edit == 2: return [(L + R0 , R1 , edit ,pw, pedit*0.8 )]
    noedit = [(L + R0 , R1 , edit ,pw, pedit*0.8 )]
    delete  =[(L , R1, edit+1,Pw(L+R1), pedit*Pedit(L[-1]+R0,L[-1]))] if len(L)>0 else [('' , R1, edit+1,Pw(R1), pedit*Pedit(R0,''))]
    replace =[(L +c, R1 , edit+1, Pw(L+c+R1), pedit*Pedit(R0,c) )for c in letters]
    insert = [(L + R0 + c , R1 ,edit+1,Pw(L + R0 + c + R1), pedit*Pedit(R0,R0+c))for c in letters]
    transpose = [( L[:-1] + R0 ,L[-1]+ R1 , edit+1,Pw(L[:-1] + R0 +L[-1]+ R1), pedit+Pedit(L[-1]+R0,R0+L[-1]))] if len(L)>0 else []
#     transpose = [( L + R1[0] ,R0+ R1[1:] , edit+1,Pw(L + R1[0] +R0+ R1[1:]), pedit+Pedit(L + R1,R0+R1[1:]))]

#     delete  =[(L , R1, edit+1,Pw(L+R1), pedit*Pedit(R0,""))]  #if len(L)>0 else []

    return noedit + delete + replace + insert + transpose

In [112]:
def candidates(word): 
    "Generate possible spelling corrections for word."
    return (known([word]) or known(edits1(word)) or known(edits2(word)) or [word])

def known(words): 
    "The subset of `words` that appear in the dictionary of WORDS."
    return set(w for w in words if w in WORDS)

def edits1(word):
    "All edits that are one edit away from `word`."
    letters    = 'abcdefghijklmnopqrstuvwxyz'
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    deletes    = [L + R[1:]               for L, R in splits if R]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
    replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters]
    inserts    = [L + c + R               for L, R in splits for c in letters]
    return set(deletes + transposes + replaces + inserts)

def edits2(word):
    "All edits that are two edits away from `word`."
    return (e2 for e1 in edits1(word) for e2 in edits1(e1))

In [113]:
count = {}
count_c = {}
for line in open('count_1.txt', 'r'):
#     print(line.split()[0],line.split()[1])
#     print(line.split('\t')[0].split('|')[0],line.split('\t')[0].split('|')[1])
    count[(line.split('\t')[0].split('|')[0],line.split('\t')[0].split('|')[1])] = int(line.split('\t')[1].strip('\n'))
    if line.split('\t')[0].split('|')[1] in count_c:
        count_c[line.split('\t')[0].split('|')[1]] += int(line.split('\t')[1].strip('\n'))
    else:
        count_c[line.split('\t')[0].split('|')[1]] = int(line.split('\t')[1].strip('\n'))


Nall = len(count)
N0 =  26*26*26*26+2*26*26*26+26*26 - Nall
All_N_Count =  Counter(count.values())
# print(All_N_Count)
Nr = [ N0 if r == 0 else All_N_Count[r] for r in range(12) ]
# print(Nr)

def Pw(word, N=sum(count_word.values())): 
#     if word in count_word:
    return count_word[word] / N
#     else:
#         return 0
#         print(10/10**len(word)/Nw)
#         return (1/1000**len(word))/N

def smooth(count, r=10):
#     print(type(count))
    r = int(r)
    if int(count) <= r:
#         if (count+1)*0.7*Nr[count+1]/Nr[count] > 1:
#             print(count,(count+1)*Nr[count+1],Nr[count])
        return (count+1)*Nr[count+1]/Nr[count]
    else:
#         print('smooth',All_N_Count[count])
#         return 0
        return All_N_Count[count]

def Pedit(w, c):
    if c in count_c:
        if count_c[c] > 0:
            if (w,c) not in count:
#                 return log(smooth(0))
#                 print('up',w,c,smooth(0),count_c[c])
                return smooth(0)/count_c[c]

            else:
                try:
#                     if smooth(count[(w,c)])/(count_c[c]) > 1:
#                         print('under',w,c, smooth(count[(w,c)]),count_c[c])
                    return smooth(count[(w,c)])/count_c[c]
                except:
                    print(count_c[c])
    else:
        return 0

# '''Combining channel probability with word probability to score states'''
def P(pedit, pw,edit):
#     return pw
    return pw*pedit/(edit+1)

In [ ]:
print(Nr)
print(Nr[1]*1*0.5/Nr[2])
print(Nr[2])
print(count_c)
print(count)

In [17]:
correction('weanter')

[('wanted', '', 2, 0.0001909312154609465, 0.0004558976840649845),
 ('walter', '', 2, 1.7927813658304835e-06, 0.00043919813519813526),
 ('webster', '', 2, 2.509893912162677e-05, 2.6699241843598322e-05),
 ('water', '', 2, 0.00016762505770515022, 3.9255202628696605e-06),
 ('weaker', '', 2, 2.1513376389965804e-05, 2.453450164293538e-05),
 ('wander', '', 2, 8.963906829152418e-06, 3.964327687521604e-05),
 ('winter', '', 2, 4.6612315511592574e-05, 3.878787878787879e-06),
 ('waiter', '', 2, 6.2747347804066925e-06, 2.8703030303030304e-05),
 ('weather', '', 2, 3.764840868244015e-05, 4.205914567360351e-06),
 ('decanter', '', 2, 1.7927813658304835e-06, 5.739607602125744e-05)]

In [186]:
hits = 4

In [296]:
import LinggleAPI
import requests
import math

SE = LinggleAPI.Linggle()
    # Search_Result = SE.search("approach that *")
    # print Search_Result
    # at * time
# test = 'when the brake is finished'.split()
raw_test = 'at brake time'

test = raw_test.lower().split()
print(test)
    # test = '? is finished'
    # test = 'brake is finished'
    
trigram = {}


for i in range(len(test) - 2):
#     print(i)
    now_tri = ' '.join(test[i:i + 3])
    res = SE[now_tri]
    if res:
        c = '\n'.join('\t'.join([str(y) for y in x]) for x in res)
        trigram[now_tri] = float(c.split('\t')[1])
#         print('\n'.join('\t'.join([str(y) for y in x]) for x in res))

    else:
        trigram[now_tri] = 0
        
        print('not found')
        
print(trigram)
can_tri = min(trigram,key = trigram.get)
# for same in sorted(trigram,key = trigram.get):
    
print(can_tri)

['at', 'brake', 'time']
not found
{'at brake time': 0}
at brake time


In [297]:
candidates = []
sen_and_err = {}
for i, t in enumerate(str(can_tri).split(' ')):
    if len(t) < 3:
        pass
    else:
#         print(t)
        for i,c in enumerate(correction(t)):
#             print(c)
            s = []
            for ot in test:
#                 print(ot)
                if str(t) == str(ot):
                    s.append(c[0]+c[1])
                else:
                    s.append(ot)
#             print(s)
#             candidates[str(t)] = 
#             candidates.append('\t'.join([str(y) for y in s]))
            candidates.append([s,c[0]+c[1],i+1])
            sen_and_err['\t'.join([str(y) for y in s])] = (str(t),c[0]+c[1])
                
#                     a[n] = 10
#             print(test[i])
#             print(i[0])
        

In [298]:
pprint(candidates)

[[['at', 'brake', 'time'], 'brake', 1],
 [['at', 'barge', 'time'], 'barge', 2],
 [['at', 'broke', 'time'], 'broke', 3],
 [['at', 'bake', 'time'], 'bake', 4],
 [['at', 'brace', 'time'], 'brace', 5],
 [['at', 'drake', 'time'], 'drake', 6],
 [['at', 'burke', 'time'], 'burke', 7],
 [['at', 'bragg', 'time'], 'bragg', 8],
 [['at', 'break', 'time'], 'break', 9],
 [['at', 'trade', 'time'], 'trade', 10],
 [['at', 'brake', 'time'], 'time', 1],
 [['at', 'brake', 'them'], 'them', 2],
 [['at', 'brake', 'tied'], 'tied', 3],
 [['at', 'brake', 'times'], 'times', 4],
 [['at', 'brake', 'lime'], 'lime', 5],
 [['at', 'brake', 'amie'], 'amie', 6],
 [['at', 'brake', 'limb'], 'limb', 7],
 [['at', 'brake', 'tide'], 'tide', 8],
 [['at', 'brake', 'him'], 'him', 9],
 [['at', 'brake', 'life'], 'life', 10]]


In [299]:

score = {}
for test in candidates:
    print(test)
    can_trigram = {}
    tri_m_sum = 1
    change_word = test[1] 
    for i in range(len(test[0]) - 2):
        now_tri = ' '.join(test[0][i:i + 3])
        res = SE.search(' '.join(test[0][i:i + 3]))
        if res:
            c = '\n'.join('\t'.join([str(y) for y in x]) for x in res)
            can_trigram[now_tri] = float(c.split('\t')[1])/test[2]
            print('\n'.join('\t'.join([str(y) for y in x]) for x in res))
        else:
            if now_tri[0] in Pdist and now_tri[1] in Pdist and now_tri[2] in Pdist:
                can_trigram[now_tri] = 1/test[2]
            else:
                can_trigram[now_tri] = 0
#                 print('not found')
            
    for k in can_trigram.values():
        tri_m_sum *= k
        
    score['\t'.join([str(y) for y in test[0]])] = tri_m_sum
#     print(sum(c for c in can_trigram.items()))

[['at', 'brake', 'time'], 'brake', 1]
[['at', 'barge', 'time'], 'barge', 2]
[['at', 'broke', 'time'], 'broke', 3]
[['at', 'bake', 'time'], 'bake', 4]
[['at', 'brace', 'time'], 'brace', 5]
[['at', 'drake', 'time'], 'drake', 6]
[['at', 'burke', 'time'], 'burke', 7]
[['at', 'bragg', 'time'], 'bragg', 8]
[['at', 'break', 'time'], 'break', 9]
at break time	4413
[['at', 'trade', 'time'], 'trade', 10]
at trade time	69
[['at', 'brake', 'time'], 'time', 1]
[['at', 'brake', 'them'], 'them', 2]
[['at', 'brake', 'tied'], 'tied', 3]
[['at', 'brake', 'times'], 'times', 4]
[['at', 'brake', 'lime'], 'lime', 5]
[['at', 'brake', 'amie'], 'amie', 6]
[['at', 'brake', 'limb'], 'limb', 7]
[['at', 'brake', 'tide'], 'tide', 8]
[['at', 'brake', 'him'], 'him', 9]
[['at', 'brake', 'life'], 'life', 10]


In [300]:
# pprint(can_trigram)
pprint(score)

{'at\tbake\ttime': 0,
 'at\tbarge\ttime': 0,
 'at\tbrace\ttime': 0,
 'at\tbragg\ttime': 0,
 'at\tbrake\tamie': 0,
 'at\tbrake\thim': 0,
 'at\tbrake\tlife': 0,
 'at\tbrake\tlimb': 0,
 'at\tbrake\tlime': 0,
 'at\tbrake\tthem': 0,
 'at\tbrake\ttide': 0,
 'at\tbrake\ttied': 0,
 'at\tbrake\ttime': 0,
 'at\tbrake\ttimes': 0,
 'at\tbreak\ttime': 490.3333333333333,
 'at\tbroke\ttime': 0,
 'at\tburke\ttime': 0,
 'at\tdrake\ttime': 0,
 'at\ttrade\ttime': 6.9}


In [301]:
print(max(score,key = score.get))
# print(score)
7433913360
255563436120
# error_sen = '\t'.join([str(y) for y in test])
# correction_sen = max(score, key=score.get)
# print(correction_sen)
# print(max(can_trigram))

at	break	time


255563436120

In [302]:
error_sen = ' '.join([str(y) for y in test[0]])
correction_sen = max(score, key=score.get)
error_text = sen_and_err.get(correction_sen)[0]
correction_text = sen_and_err.get(correction_sen)[1]

In [303]:
print("Error:",error_text)
print("Candidates:")
for i in correction(error_text):
    print(i[0])
print("Correction:",correction_text)
correction_sen = " ".join(i for i in correction_sen.split('\t'))
print(raw_test ,'--->',correction_sen)
if testing_dict.get(raw_test.lower()) == correction_sen:
    hits += 1
print("Hits:",hits)
print('\n')

Error: brake
Candidates:
brake
barg
broke
bake
brace
drake
burke
bragg
break
trade
Correction: break
at brake time ---> at break time
Hits: 10




In [295]:
print(testing_dict.get(raw_test.lower()) )
for i in correction_sen.split('\t'):
    print(i)
correction_sen = " ".join(i for i in correction_sen.split('\t'))
print(correction_sen)
if correction_sen == testing_dict.get(raw_test.lower()):
    print('tes')

an ancient method of hunting
an action method of hunting
an action method of hunting


In [139]:
testing_data = open('lab4.test.1.txt')
testing_dict = {}
for i in testing_data.readlines()[:20]:
    testing_dict[i.split('\t')[0].lower().strip()] = i.split('\t')[1].lower().strip()

In [ ]:
for i in testing_dict:
    print(i,'|',testing_dict.get(i))
print(testing_dict.get('i felt very strang'))

In [ ]:
fusion_word = open('lab4.confusables.txt')
fusion_dict = {}
for i in fusion_word.readlines():
    fusion_dict[i.split('\t')[0].lower().strip()] = i.split('\t')[1].lower().strip()

In [ ]:
hits = 0
import NetSpeakAPI
SE = NetSpeakAPI.NetSpeak()
for i in testing_dict:

# def CORRECTION(raw_test):
    global hits
    raw_test = i
    
#     raw_test = 'I felt very strang'
    test = raw_test.lower().split()
    
    trigram = {}


    for i in range(len(test) - 2):
        now_tri = ' '.join(test[i:i + 3])
        res = SE.search(' '.join(test[i:i + 3]))
        if res:
            c = '\n'.join(' '.join([str(y) for y in x]) for x in res)
#             print(c)
            trigram[now_tri] = float(c.split(' ')[-1])
#         print('\n'.join('\t'.join([str(y) for y in x]) for x in res))
        else:
            trigram[now_tri] = 0
#             print('not found')
    
#     print(trigram)
    can_tri = min(trigram,key=trigram.get)
    
    candidates = []
    sen_and_err = {}
    for i, t in enumerate(str(can_tri).split(' ')):
        if len(t) < 3:
            pass
        else:
            for c in correction(t):
                s = []

                for ot in test:
                    if str(t) == str(ot):
                        s.append(c[0]+c[1])
                    else:
                        s.append(ot)
#                 print(s)
#             candidates[str(t)] = 
#             candidates.append('\t'.join([str(y) for y in s]))
                candidates.append(s)
                sen_and_err[' '.join([str(y) for y in s])] = (str(t),c[0]+c[1])
        
    score = {}
#     print(candidates)
    for test in candidates:
#         print(test)
        can_trigram = {}
        m_sum = 1
        for i in range(len(test) - 2):
            now_tri = ' '.join(test[i:i + 3])
            res = SE.search(' '.join(test[i:i + 3]))
            if res:
                c = '\n'.join(' '.join([str(y) for y in x]) for x in res)
                can_trigram[now_tri] = float(c.split(' ')[-1])
#                 print('\n'.join(' '.join([str(y) for y in x]) for x in res))
            else:
                can_trigram[now_tri] = 0
#                 print('not found')
    
        for k in can_trigram.values():
            m_sum *= k
        score[' '.join([str(y) for y in test])] = m_sum
    
    error_sen = ' '.join([str(y) for y in test])
    correction_sen = max(score, key=score.get)
    error_text = sen_and_err.get(correction_sen)[0]
    correction_text = sen_and_err.get(correction_sen)[1]
    
#     for i, t in enumerate(str(can_tri).split(' ')):
#         if t in fusion_dict:
#             correction_text = fusion_dict[t]
#             error_text = t
#             for n, token in enumerate(raw_test.lower()):
#                 if token == t:
#                     raw_test.lower()[n] = correction_text
#             break
    
    print("Error:",error_text)
    print("Candidates:")
    for i in correction(error_text):
        print(i[0])
    print("Correction:",correction_text)
    print(raw_test ,'--->',correction_sen)
    if testing_dict.get(raw_test.lower()) == correction_sen:
        hits += 1
    print(hits)

In [ ]:
for i in testing_dict:
    CORRECTION(i)

In [ ]:
for i in testing_dict:
    raw_test = i
    test = raw_test.lower().split()
    print(test)
    # test = '? is finished'
    # test = 'brake is finished'
    
    trigram = {}


    for i in range(len(test) - 2):
        now_tri = ' '.join(test[i:i + 3])
        res = SE[now_tri]
        if res:
            c = '\n'.join('\t'.join([str(y) for y in x]) for x in res)
            trigram[now_tri] = float(c.split('\t')[1])
#         print('\n'.join('\t'.join([str(y) for y in x]) for x in res))

        else:
            trigram[now_tri] = 0
            print('not found')
        
    print(trigram)
    can_tri = min(trigram,key = trigram.get) 
    print(can_tri)